## Virtual Product creation to test C3 datacube stats

In [ ]:
# export PYTHONPATH=/g/data/r78/cb3058/dea-notebooks/vegetation_anomalies/dc_refactor/datacube-stats/:$PYTHONPATH

In [32]:
from datacube.virtual import construct_from_yaml
from datacube import Datacube
from datacube.drivers.netcdf import create_netcdf_storage_unit, write_dataset_to_netcdf
from dask.distributed import LocalCluster, Client
import sys
sys.path.append('../../Scripts')
from dea_datahandling import load_ard
import numpy as np
import xarray as xr

In [ ]:
client = locals().get('client', None)
if client is not None:
    client.close()
    del client
    
# client = Client(n_workers=6, threads_per_worker=1, memory_limit='4GB')
# client

In [2]:
dc = Datacube(env='c3-samples')

In [4]:
# dc.list_products()

In [20]:
lat, lon = -33.2, 149.1
buffer = 0.05

time = ('1987', '1999')

query = {'lon': (lon - buffer, lon + buffer),
         'lat': (lat - buffer, lat + buffer),
         'time': time,
         'measurements':['nbart_red', 'nbart_nir', 'fmask', 'oa_nbart_contiguity'],
         'output_crs' :'EPSG:3577',
         'resolution' : (-30, 30),
          'group_by' :'solar_day',
        'dask_chunks': {'x': 250, 'y':250}}
    

In [21]:
ds = dc.load(product= 'ga_ls5t_ard_3', **query)

In [76]:

    
quarter= {'JFM': [1,2,3],
                       'FMA': [2,3,4],
                       'MAM': [3,4,5],
                       'AMJ': [4,5,6],
                       'MJJ': [5,6,7],
                       'JJA': [6,7,8],
                       'JAS': [7,8,9],
                       'ASO': [8,9,10],
                       'SON': [9,10,11],
                       'OND': [10,11,12],
                       'NDJ': [11,12,1],
                       'DJF': [12,1,2],
                      }

def compute(data):

    def attrs_reassign(da, dtype=np.float32):
        da_attr = data.attrs
        da = da.assign_attrs(**da_attr)
        return da

    ndvi = xr.Dataset(data_vars={'ndvi': (data.nbart_nir - data.nbart_red) / (data.nbart_nir + data.nbart_red)},
                          coords=data.coords,
                          attrs=dict(crs=data.crs))
    
    ndvi_var = []
    for q in quarter:
        ix=ndvi['time.month'].isin(quarter[q])
        ndvi_clim_mean=ndvi.where(ix,drop = True).mean(dim='time')   
        ndvi_clim_mean=ndvi_clim_mean.to_array(name='ndvi_clim_mean_'+q).drop('variable').squeeze()
        ndvi_var.append(ndvi_clim_mean)

    q_clim_mean = xr.merge(ndvi_var)   

    #assign back attributes
    q_clim_mean.attrs = data.attrs 
    q_clim_mean = q_clim_mean.apply(attrs_reassign, keep_attrs=True)  

    return q_clim_mean

In [77]:
compute(ds)

<xarray.Dataset>
Dimensions:            (x: 356, y: 411)
Coordinates:
  * y                  (y) float64 -3.721e+06 -3.721e+06 ... -3.733e+06
  * x                  (x) float64 1.573e+06 1.574e+06 ... 1.584e+06 1.584e+06
Data variables:
    ndvi_clim_meanJFM  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_meanFMA  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_meanMAM  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_meanAMJ  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_meanMJJ  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_meanJJA  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_meanJAS  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_meanASO  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_meanSON  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_meanOND  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_meanNDJ  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_meanDJF  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
Attributes:
    crs:      EPSG:3577

In [ ]:
ndvi_clim_mean = construct_from_yaml("""
        aggregate: datacube_stats.external.ndvi_clim_mean
        input:
          reproject:
            output_crs: EPSG:3577
            resolution: [-30, 30]
            resampling: average
          input:
            collate:
              - transform: apply_mask
                mask_measurement_name: fmask
                dilation: 3
                input:
                  transform: expressions
                  output: 
                    fmask:
                        formula: (fmask != 2) & (fmask != 3) & (fmask != 0) & (oa_nbart_contiguity == 1)
                        nodata: False
                    nbart_red: nbart_red
                    nbart_nir: nbart_nir
                  input:
                    product: ga_ls8c_ard_3
                    measurements: [nbart_red, nbart_nir, fmask, oa_nbart_contiguity]
                    gqa_iterative_mean_xy: [0, 1]
                    dataset_predicate: datacube_stats.main.ls8_on
              - transform: apply_mask
                mask_measurement_name: fmask
                dilation: 3
                input:
                  transform: expressions
                  output: 
                    fmask:
                        formula: (fmask != 2) & (fmask != 3) & (fmask != 0) & (oa_nbart_contiguity == 1)
                        nodata: False
                    nbart_red: nbart_red
                    nbart_nir: nbart_nir
                  input:
                    product: ga_ls7c_ard_3
                    measurements: [nbart_red, nbart_nir, fmask, oa_nbart_contiguity]
                    gqa_iterative_mean_xy: [0, 1]
                    dataset_predicate: datacube_stats.main.ls7_on
              - transform: apply_mask
                mask_measurement_name: fmask
                dilation: 3
                input:
                  transform: expressions
                  output: 
                    fmask:
                        formula: (fmask != 2) & (fmask != 3) & (fmask != 0) & (oa_nbart_contiguity == 1)
                        nodata: False
                    nbart_red: nbart_red
                    nbart_nir: nbart_nir
                  input:
                    product: ga_ls5c_ard_3
                    measurements: [nbart_red, nbart_nir,fmask, oa_nbart_contiguity]
                    gqa_iterative_mean_xy: [0, 1]
                    dataset_predicate: datacube_stats.main.ls5_on
    """)

In [ ]:
datasets = ndvi_clim_mean.query(dc, **query)
grouped = ndvi_clim_mean.group(datasets, **query)
results = ndvi_clim_mean.fetch(grouped, **query, dask_chunks={'time':-1, 'x':250, 'y':250})
results.load()
